In [29]:
from m3_learning.be.filters import clean_interpolate
import numpy as np
import pytest
from scipy.interpolate import CubicSpline
import sys
sys.path.append('/home/ferroelectric/Documents/m3_learning/m3_learning/src')


In [11]:
arr1 = np.array([[1, 2, np.nan, 4], [5, np.nan, 7, 8],
                [5, 6, 7, 8], [5, 6, 7, 8]])
expected1 = np.array(
    [[1, 2, 3, 4], [5, 4.66666667, 7, 8], [5, 6, 7, 8], [5, 6, 7, 8]])

print(clean_interpolate(arr1, axis=0))
assert np.allclose(clean_interpolate(arr1, axis=0), expected1)

[[1.         2.         7.         4.        ]
 [5.         4.66666667 7.         8.        ]
 [5.         6.         7.         8.        ]
 [5.         6.         7.         8.        ]]


AssertionError: 

In [13]:
arr9 = np.array([[[1, 2, np.nan, 4], [5, np.nan, 7, 8]], [
                    [9, 10, np.nan, 12], [13, np.nan, 15, 16]]])
expected9 = np.array([[[1, 2, 3, 4], [5, 6, 7, 8]], [
                        [9, 10, 11, 12], [13, 14, 15, 16]]])
clean_interpolate(arr9, axis=1)

array([[[ 1.,  2., nan,  4.],
        [ 5., nan,  7.,  8.]],

       [[ 9., 10., nan, 12.],
        [13., nan, 15., 16.]]])

In [1]:
import sys

sys.path.append("../../")
# sys.path.append("/home/ferroelectric/Documents/m3_learning/m3_learning/src")
sys.path.append('../../src')

%load_ext autoreload
%autoreload 2

import numpy as np
from m3_learning.be.dataset import BE_Dataset
from m3_learning.viz.printing import printer
from m3_learning.be.nn import SHO_fit_func_nn, SHO_Model
from m3_learning.util.file_IO import download_and_unzip

2024-06-22 11:31:45.373507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-22 11:31:45.903639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/cv2/../../lib64:
2024-06-22 11:31:45.903686: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ferroelectric/micromamba/envs/paper/

In [11]:
data_path = "/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/AD063_SSO_VertBEPS_2X2um2_10X10grid_8V_0010.h5"

# instantiate the dataset object
dataset = BE_Dataset(data_path)

# print the contents of the file
dataset.print_be_tree()

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
Datasets and datagroups within the file:
------------------------------------
/
/Measurement_000
/Measurement_000/Channel_000
/Measurement_000/Channel_000/Bin_FFT
/Measurement_000/Chann

In [13]:
13209600/400/384

86.0

In [7]:
# Ensure python 3 compatibility
from __future__ import division, print_function, absolute_import
import sidpy
import pyUSID as usid
from BGlib import be as belib

# Import necessary libraries:
# General utilities:
import sys
import os
import math

# Computation:
import numpy as np
import h5py

# Visualization:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import ipywidgets as widgets

from IPython.display import display, HTML

# Finally, BGlib itself
sys.path.append('..')


# Make Notebook take up most of page width
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [8]:
input_file_path = "/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/AD063_SSO_VertBEPS_2X2um2_10X10grid_8V_0010.h5"

(data_dir, filename) = os.path.split(input_file_path)

if input_file_path.endswith('.h5'):
    # No translation here
    h5_path = input_file_path
    force = False  # Set this to true to force patching of the datafile.
    tl = belib.translators.LabViewH5Patcher()
    tl.translate(h5_path, force_patch=force)
else:
    # Set the data to be translated
    data_path = input_file_path

    (junk, base_name) = os.path.split(data_dir)

    # Check if the data is in the new or old format.  Initialize the correct translator for the format.
    if base_name == 'newdataformat':
        (junk, base_name) = os.path.split(junk)
        translator = belib.translators.BEPSndfTranslator(max_mem_mb=max_mem)
    else:
        translator = belib.translators.BEodfTranslator(max_mem_mb=max_mem)
    if base_name.endswith('_d'):
        base_name = base_name[:-2]
    # Translate the data

    h5_path = translator.translate(
        data_path, show_plots=True, save_plots=False)

h5_file = h5py.File(h5_path, 'r+')
print('Working on:\n' + h5_path)

h5_main = usid.hdf_utils.find_dataset(h5_file, 'Raw_Data')

File is already Pycroscopy ready.
Working on:
/home/ferroelectric/Documents/m3_learning/m3_learning/src/m3_learning/tests/be/AD063_SSO_VertBEPS_2X2um2_10X10grid_8V_0010.h5


/home/ferroelectric/micromamba/envs/paper/lib/python3.10/site-packages/sidpy/sid/translator.py:42: FutureWarning: Consider using sidpy.Reader instead of sidpy.Translator if possible and contribute your reader to ScopeReaders
  warn('Consider using sidpy.Reader instead of sidpy.Translator if '


In [9]:
print('Datasets and datagroups within the file:\n------------------------------------')
usid.hdf_utils.print_tree(h5_file)

print('\nThe main dataset:\n------------------------------------')
print(h5_main)
print('\nThe ancillary datasets:\n------------------------------------')
print(h5_file['/Measurement_000/Channel_000/Position_Indices'])
print(h5_file['/Measurement_000/Channel_000/Position_Values'])
print(h5_file['/Measurement_000/Channel_000/Spectroscopic_Indices'])
print(h5_file['/Measurement_000/Channel_000/Spectroscopic_Values'])

print('\nMetadata or attributes in a datagroup\n------------------------------------')
for key in h5_file['/Measurement_000'].attrs:
    print('{} : {}'.format(key, h5_file['/Measurement_000'].attrs[key]))

Datasets and datagroups within the file:
------------------------------------
/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices

The main dataset:
------------------------------------
[<HDF5 dataset "Raw_Data": shape (100, 45312), type

In [21]:
h5_file["Measurement_000/Channel_000/UDVS"]

<HDF5 dataset "UDVS": shape (768, 7), type "<f4">

In [22]:
h5_file['/Measurement_000/Channel_000/Spectroscopic_Values/UDVS_Indices']

KeyError: 'Unable to synchronously open object (message type not found)'

In [4]:
33024/(786)

42.01526717557252

In [10]:
h5_file["Measurement_000"].attrs["VS_steps_per_full_cycle"]*h5_file["Measurement_000"].attrs["VS_number_of_cycles"] * \
    (2 if h5_file["Measurement_000"].attrs["VS_measure_in_field_loops"]
     == 'in and out-of-field' else 1)

768

In [7]:
63360/165

384.0